# Train LLM from scratch

## Train a tokenizer

In [1]:
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

In [2]:
VOCAB_SIZE = 4000
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
MODEL_DIR = 'gpt'

In [3]:
paths = [str(x) for x in Path(".").glob("**/*.txt")]
tokenizer = ByteLevelBPETokenizer()
tokenizer.enable_padding(pad_token=PAD_TOKEN)
tokenizer.unk_token = UNK_TOKEN
tokenizer.train(files="ss.txt", vocab_size=VOCAB_SIZE, min_frequency=2, special_tokens=[PAD_TOKEN, UNK_TOKEN])

In [4]:
tokenizer.encode("hello my name is").tokens

['he', 'll', 'o', 'Ġmy', 'Ġname', 'Ġis']

In [5]:
!mkdir {MODEL_DIR}
tokenizer.save_model(f"{MODEL_DIR}")

['gpt\\vocab.json', 'gpt\\merges.txt']

In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_DIR, pad_token=PAD_TOKEN, unk_token=UNK_TOKEN)

c:\Users\saisa\.conda\envs\llms\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Train a Language Model from scratch

### Setup the model

In [7]:
import torch

In [8]:
torch.cuda.is_available()

True

In [9]:
from transformers import OpenAIGPTConfig
gpt_config = OpenAIGPTConfig(
    vocab_size=VOCAB_SIZE,
    n_embd=64,
    n_head=8,
    n_layer=10,
    n_positions=512
)

In [10]:
from transformers import OpenAIGPTLMHeadModel
model = OpenAIGPTLMHeadModel(config=gpt_config)

In [11]:
model.num_parameters()

788608

### Build dataset

In [12]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./ss.txt",
    block_size=128
)

c:\Users\saisa\.conda\envs\llms\lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

### Setup trainer

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt",
    overwrite_output_dir=True,
    num_train_epochs=500,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=3,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [15]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
  3%|▎         | 505/17000 [00:20<10:39, 25.78it/s]

{'loss': 7.5573, 'grad_norm': 0.8311532139778137, 'learning_rate': 4.8529411764705885e-05, 'epoch': 14.71}


  6%|▌         | 1003/17000 [00:41<10:13, 26.09it/s]

{'loss': 6.6074, 'grad_norm': 0.720300555229187, 'learning_rate': 4.705882352941177e-05, 'epoch': 29.41}


  9%|▉         | 1504/17000 [01:01<10:03, 25.68it/s]

...

 97%|█████████▋| 16505/17000 [11:12<00:18, 26.38it/s]

{'loss': 4.5888, 'grad_norm': 1.8762872219085693, 'learning_rate': 1.4705882352941177e-06, 'epoch': 485.29}


100%|██████████| 17000/17000 [11:32<00:00, 24.55it/s]

{'loss': 4.5851, 'grad_norm': 2.2988784313201904, 'learning_rate': 0.0, 'epoch': 500.0}
{'train_runtime': 692.5469, 'train_samples_per_second': 1555.851, 'train_steps_per_second': 24.547, 'train_loss': 5.19054292566636, 'epoch': 500.0}
CPU times: total: 3min 50s
Wall time: 11min 32s


TrainOutput(global_step=17000, training_loss=5.19054292566636, metrics={'train_runtime': 692.5469, 'train_samples_per_second': 1555.851, 'train_steps_per_second': 24.547, 'total_flos': 50486196245760.0, 'train_loss': 5.19054292566636, 'epoch': 500.0})

In [16]:
trainer.save_model('./gpt')

In [17]:
from transformers import pipeline
pipeline("text-generation", model=model, tokenizer=tokenizer, device='cuda')("In loving thee")

c:\Users\saisa\.conda\envs\llms\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': "In loving thee to the world'st thou art, and thy self.     "}]